<a href="https://colab.research.google.com/github/everestso/summer25/blob/main/LLM1b_DBAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DB Agent

In [ ]:
# pips

!pip install aisuite
!pip -q install "aisuite[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring_parser 0.17.0
    Uninstalling docstring_parser-0.17.0:
      Successfully uninstalled docstring_parser-0.17.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.61.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip -q install SQLAlchemy psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 70.3 MB/s eta 0:00:00


In [ ]:
# Keys
import os
import json
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API')
STUDENT_DB_URL = userdata.get('NEON_CON')

In [ ]:
# LLM as a Client

import aisuite as ai

# Create an instance of the AISuite client
client = ai.Client()

In [ ]:
schema = '''

-- #Create a relation for the 'product' information.
create table IF NOT EXISTS product (maker char(1), model int, type varchar(10));

-- # 3rd Edition Fig 2.21a
create table IF NOT EXISTS pc (model int, speed float, ram int, hd int, price int);

-- #Fig 2.21b
create table IF NOT EXISTS laptop (model int, speed float, ram int, hd int, screen float,
                                   price int);

-- # Fig 2.21c
create table IF NOT EXISTS printer (model int, color boolean, type varchar(10), price int);
);


'''

In [ ]:
# Message structure
prompt_text_schema = '''
Given the following schema:
'''

prompt_text_query = '''
Generate legal SQL to answer the following question and when doing that, only return the properly formatted as text,  do not include the designation "sql":
'''


In [ ]:
product_sql_questions = [
    "What PC models have a speed of at least 3.00?",
    "Which manufacturers make laptops with a hard disk of at least 100GB?",
    "Find the model number and price of all products (of any type) made by manufacturer B.",
    "Find the model numbers of all color laser printers.",
    "Find those manufacturers that sell laptops, but not PCs.",
    "Find those hard-disk sizes that occur in two or more PCs.",
    "Write SQL to find Maker, Model, Screen Size, and Price for laptops with screens larger than 15 inches."
]

product_sql_solutions = [
    # 1) What PC models have a speed of at least 3.00?
    "SELECT model FROM PC WHERE speed >= 3.00;",

    # 2) Which manufacturers make laptops with a hard disk of at least 100GB?
    "SELECT DISTINCT p.maker "
    "FROM Product p "
    "JOIN Laptop l ON p.model = l.model "
    "WHERE l.hd >= 100;",

    # 3) Find the model number and price of all products (of any type) made by manufacturer B.
    # (Union across subtype tables; uses textbook assumption that every product row appears in exactly one subtype table)
    "SELECT pc.model, pc.price FROM PC pc WHERE pc.model IN (SELECT model FROM Product WHERE maker = 'B') "
    "UNION ALL "
    "SELECT l.model, l.price FROM Laptop l WHERE l.model IN (SELECT model FROM Product WHERE maker = 'B') "
    "UNION ALL "
    "SELECT pr.model, pr.price FROM Printer pr WHERE pr.model IN (SELECT model FROM Product WHERE maker = 'B') "
    "ORDER BY model;",

    # 4) Find the model numbers of all color laser printers.
    "SELECT model FROM Printer WHERE color = TRUE AND type = 'laser';",

    # 5) Find those manufacturers that sell laptops, but not PCs.
    "SELECT DISTINCT maker FROM Product WHERE type = 'laptop' "
    "AND maker NOT IN (SELECT maker FROM Product WHERE type = 'pc');",

    # 6) Find those hard-disk sizes that occur in two or more PCs.
    "SELECT hd FROM PC GROUP BY hd HAVING COUNT(*) >= 2;",

    # 7) Maker, Model, Screen Size, and Price for laptops with screens larger than 15 inches.
    "SELECT p.maker, l.model, l.screen, l.price "
    "FROM Product p JOIN Laptop l ON p.model = l.model "
    "WHERE l.screen > 15;"
]



In [ ]:
question_no = 2
prompt = prompt_text_schema + schema + "\n\n" + prompt_text_query + product_sql_questions[question_no]

messages = [
    {
        "role": "user",
        "content": prompt,
    }
]

In [ ]:
response = client.chat.completions.create(
    model="groq:llama-3.3-70b-versatile",
    messages=messages
)

# See the LLM response
print(response.choices[0].message.content)
query = response.choices[0].message.content

SELECT T1.model, T2.price 
FROM product AS T1 
INNER JOIN pc AS T2 
ON T1.model = T2.model 
WHERE T1.maker = 'B'

UNION

SELECT T1.model, T2.price 
FROM product AS T1 
INNER JOIN laptop AS T2 
ON T1.model = T2.model 
WHERE T1.maker = 'B'

UNION

SELECT T1.model, T2.price 
FROM product AS T1 
INNER JOIN printer AS T2 
ON T1.model = T2.model 
WHERE T1.maker = 'B'


In [ ]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

#STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: quick row counts
with engine.connect() as conn:
    query_results = pd.read_sql_query(text(query), conn)
    query_solution_results = pd.read_sql_query(text(product_sql_solutions[question_no]), conn)

print("AI:")
print(query_results)
print("Solution:")
print(query_solution_results)

AI:
   model  price
0   2007   1429
1   1004    649
2   1005    630
3   1006   1049
Solution:
   model  price
0   1004    649
1   1005    630
2   1006   1049
3   2007   1429


In [ ]:
print (prompt)


Given the following schema:


-- #Create a relation for the 'product' information.
create table IF NOT EXISTS product (maker char(1), model int, type varchar(10));

-- # 3rd Edition Fig 2.21a
create table IF NOT EXISTS pc (model int, speed float, ram int, hd int, price int);

-- #Fig 2.21b
create table IF NOT EXISTS laptop (model int, speed float, ram int, hd int, screen float,
                                   price int);

-- # Fig 2.21c
create table IF NOT EXISTS printer (model int, color boolean, type varchar(10), price int);
);





Generate legal SQL to answer the following question and when doing that, only return the properly formatted as text,  do not include the designation "sql":
Find the model number and price of all products (of any type) made by manufacturer B.
